# CUDA-Accelerated HRP Portfolio Optimization (24-Month Lookback)

This notebook implements GPU-accelerated Hierarchical Risk Parity (HRP) portfolio optimization with **SciPy single-linkage clustering** for all quarterly rebalances from 1980 to present using a **24-month lookback window**.

**GPU Acceleration:**
- **Covariance matrix computation** (Ledoit-Wolf shrinkage estimation)
- **Correlation matrix calculation** (vectorized operations)
- **Distance matrix computations** (correlation distance + Euclidean distance)
- **Cumulative return calculations**

**CPU Operations:**
- **Hierarchical clustering** via `scipy.cluster.hierarchy.linkage(method='single')` with precomputed distance matrix
- **Recursive bisection** for weight allocation
- **Data I/O operations**

**Processing Scope:**
- All quarterly rebalance dates in the dataset (typically 150+ windows)
- **24-month rolling window** for each rebalance (lookback period)
- Minimum 20 stocks per portfolio after filtering

**Requirements:**
- NVIDIA GPU with CUDA support (compute capability 6.0+)
- `cupy-cuda11x` or `cupy-cuda12x` for GPU operations
- `scikit-learn` for Ledoit-Wolf covariance
- `scipy` for hierarchical clustering
- `pandas`, `numpy`, `tqdm`, `matplotlib`

**Output:**
- CSV file with HRP weights for all quarterly rebalances: `hrp_weights_24m_lookback.csv`
- Performance comparison plots vs CRSP value-weighted and equal-weighted benchmarks
- Detailed timing statistics for GPU vs CPU operations

**Note:** Automatically falls back to CPU-only mode if GPU unavailable.

In [1]:
# Cell 0: Install Dependencies (Optional - run if packages not installed)

# For CUDA 11.x
# !pip install cupy-cuda11x scikit-learn scipy tqdm pandas numpy matplotlib

# For CUDA 12.x
# !pip install cupy-cuda12x scikit-learn scipy tqdm pandas numpy matplotlib

import subprocess
try:
    cuda_version = subprocess.check_output(['nvcc', '--version']).decode('utf-8')
    print("CUDA Version Info:")
    print(cuda_version)
    
    if 'release 11' in cuda_version:
        print("\n✓ Detected CUDA 11.x")
        print("Install: !pip install cupy-cuda11x scikit-learn scipy tqdm pandas numpy matplotlib")
    elif 'release 12' in cuda_version:
        print("\n✓ Detected CUDA 12.x")
        print("Install: !pip install cupy-cuda12x scikit-learn scipy tqdm pandas numpy matplotlib")
    else:
        print("\n⚠ Unknown CUDA version")
except:
    print("Cannot detect CUDA version via nvcc. Checking with nvidia-smi...")
    try:
        nvidia_smi = subprocess.check_output(['nvidia-smi']).decode('utf-8')
        print(nvidia_smi)
        print("\nCheck the 'CUDA Version' in the output above.")
    except:
        print("⚠ Could not detect GPU. Will run in CPU-only mode.")

CUDA Version Info:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


✓ Detected CUDA 11.x
Install: !pip install cupy-cuda11x scikit-learn scipy tqdm pandas numpy matplotlib


In [2]:
# Cell 1: Imports and GPU Setup

import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
from sklearn.covariance import LedoitWolf
from tqdm import tqdm
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import matplotlib.pyplot as plt
import random

try:
    import cupy as cp
    GPU_AVAILABLE = True
    
    device = cp.cuda.Device()
    props = cp.cuda.runtime.getDeviceProperties(device.id)
    gpu_name = props['name'].decode('utf-8')
    total_mem = props['totalGlobalMem'] / 1e9
    cuda_version = cp.cuda.runtime.runtimeGetVersion()
    
    print(f"✓ GPU Detected: {gpu_name}")
    print(f"  CUDA Version: {cuda_version}")
    print(f"  Memory: {total_mem:.2f} GB")
except ImportError as e:
    print(f"⚠ GPU libraries not available: {e}")
    print("  Running in CPU-only mode")
    GPU_AVAILABLE = False
    cp = np

data_path = r'ADA-HRP-Preprocessed-DATA.csv'
rolling_dir = r'Rolling Windows'
os.makedirs(rolling_dir, exist_ok=True)

print(f"\nMode: {'GPU (CUDA)' if GPU_AVAILABLE else 'CPU'}")
print(f"Output Directory: {rolling_dir}")

✓ GPU Detected: NVIDIA L40
  CUDA Version: 11080
  Memory: 47.58 GB

Mode: GPU (CUDA)
Output Directory: Rolling Windows


In [3]:
# Cell 2: Load Data and Prepare Quarterly Rebalance Dates

df = pd.read_csv(data_path)
print(f"Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")

date_cols = [col for col in df.columns if col not in ['PERMNO', 'Company_Ticker']]

parsed_strs = [col.replace('_', ':') for col in date_cols]
parsed_dates = pd.to_datetime(parsed_strs, errors='coerce')

sort_order = np.argsort(parsed_dates)
date_cols = [date_cols[i] for i in sort_order]
dates = parsed_dates[sort_order]
date_strs = [d.strftime('%Y-%m-%d') for d in dates]

for col in date_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

stocks_df = df[df['PERMNO'].notna()].copy()
print(f"Stocks: {len(stocks_df)} securities")

def get_quarterly_dates(dates):
    quarterly_dates = []
    df_dates = pd.DataFrame({'date': dates})
    df_dates['year'] = df_dates['date'].dt.year
    df_dates['quarter'] = df_dates['date'].dt.quarter
    quarterly_ends = df_dates.groupby(['year', 'quarter'])['date'].max()
    return quarterly_ends.tolist()

quarterly_rebalance_dates = get_quarterly_dates(dates)
print(f"Quarterly rebalance dates: {len(quarterly_rebalance_dates)}")

Loaded data: 33883 rows, 542 columns
Stocks: 33881 securities
Quarterly rebalance dates: 180


In [4]:
# Cell 3: Define HRP Functions

def get_correlation_distance_gpu(corr_gpu):
    """Compute correlation distance matrix on GPU"""
    corr_gpu = cp.clip(corr_gpu, -1.0, 1.0)
    dist = cp.sqrt(cp.clip((1 - corr_gpu) / 2, 0.0, None))
    
    if cp.any(~cp.isfinite(dist)):
        print("⚠ WARNING: NaN/Inf detected in correlation distance matrix")
    
    dist = cp.nan_to_num(dist, nan=0.5, posinf=0.5, neginf=0.5)
    return dist

def get_euclidean_distance_gpu(dist_gpu):
    """Compute pairwise Euclidean distances on GPU"""
    n = dist_gpu.shape[0]
    squared_norms = cp.sum(dist_gpu ** 2, axis=1, keepdims=True)
    eucl_dist = cp.sqrt(cp.clip(squared_norms + squared_norms.T - 2 * cp.dot(dist_gpu, dist_gpu.T), 0.0, None))
    
    if cp.any(~cp.isfinite(eucl_dist)):
        print("⚠ WARNING: NaN/Inf detected in Euclidean distance matrix")
    
    eucl_dist = cp.nan_to_num(eucl_dist, nan=1e-4, posinf=1e-4, neginf=1e-4)
    return eucl_dist

def compute_covariance_gpu(returns_np, returns_gpu=None):
    """Compute shrunk covariance using Ledoit-Wolf"""
    lw = LedoitWolf()
    lw.fit(returns_np)
    cov_array = lw.covariance_
    shrinkage = lw.shrinkage_
    
    if GPU_AVAILABLE:
        cov_gpu = cp.asarray(cov_array)
        return cov_array, shrinkage, cov_gpu
    else:
        return cov_array, shrinkage, None

def get_quasi_diag(link):
    """Seriation from hierarchical clustering linkage"""
    link = link.astype(int)
    sort_ix = pd.Series([link[-1, 0], link[-1, 1]])
    num_items = link[-1, 3]
    while sort_ix.max() >= num_items:
        sort_ix.index = range(0, sort_ix.shape[0] * 2, 2)
        df0 = sort_ix[sort_ix >= num_items]
        i = df0.index
        j = df0.values - num_items
        sort_ix[i] = link[j, 0]
        df0 = pd.Series(link[j, 1], index=i + 1)
        sort_ix = pd.concat([sort_ix, df0])
        sort_ix = sort_ix.sort_index()
        sort_ix.index = range(sort_ix.shape[0])
    return sort_ix.tolist()

def perform_single_linkage_clustering(eucl_dist_condensed):
    """Single-linkage hierarchical clustering using SciPy"""
    return sch.linkage(eucl_dist_condensed, method='single')

def get_cluster_var(cov, c_items):
    """Compute cluster variance using inverse-variance portfolio"""
    cov_ = cov.loc[c_items, c_items].values
    
    if len(c_items) == 1:
        return cov_[0, 0]
    
    variances = np.diag(cov_)
    variances = np.maximum(variances, 1e-10)
    
    ivp = 1.0 / variances
    ivp /= ivp.sum()
    
    w_ = ivp.reshape(-1, 1)
    cVar = (w_.T @ cov_ @ w_)[0, 0]
    
    return cVar

def get_recursive_bisection(cov, sort_ix, debug=False):
    """Recursive bisection for HRP weight allocation"""
    w = pd.Series(1.0, index=sort_ix)
    c_items = [sort_ix]
    
    iteration = 0
    while len(c_items) > 0:
        c_items = [i[j:k] for i in c_items 
                   for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) 
                   if len(i) > 1]
        
        if len(c_items) == 0:
            break
            
        for i in range(0, len(c_items), 2):
            if i + 1 >= len(c_items):
                break
                
            c_items0 = c_items[i]
            c_items1 = c_items[i + 1]
            
            c_var0 = get_cluster_var(cov, c_items0)
            c_var1 = get_cluster_var(cov, c_items1)
            
            alpha = 1 - c_var0 / (c_var0 + c_var1)
            
            if debug and iteration < 5:
                print(f"  Iteration {iteration}, pair {i//2}: var0={c_var0:.6f}, var1={c_var1:.6f}, alpha={alpha:.6f}")
            
            w[c_items0] *= alpha
            w[c_items1] *= 1 - alpha
            
        iteration += 1
    
    w = w / w.sum()
    
    if debug:
        print(f"  Total iterations: {iteration}")
        print(f"  Final weight sum: {w.sum():.10f}")
        print(f"  Weight range: [{w.min():.6f}, {w.max():.6f}]")
    
    return w

print("✓ HRP functions defined")

✓ HRP functions defined


In [ ]:
# Cell 4: Process All Quarterly Rebalances (24-Month Lookback)

lookback_months = 24
weights_filename = 'hrp_weights_24m_lookback.csv'
weights_output_path = os.path.join(rolling_dir, weights_filename)
target_rebalance_dates = quarterly_rebalance_dates
print(f"Processing all {len(target_rebalance_dates)} quarterly rebalance dates (24-month lookback)")

weights_list = []
timing_stats = {'cov': [], 'corr': [], 'dist': [], 'cluster': [], 'weights': [], 'io': [], 'total': []}
skipped_count = 0

for rebal_date in tqdm(target_rebalance_dates, desc="Processing quarterly rebalances"):
    t_start = time.time()
    t_io_start = time.time()
    rebal_str = rebal_date.strftime('%Y-%m-%d')
    
    try:
        rebal_idx = date_strs.index(rebal_str)
    except ValueError:
        skipped_count += 1
        continue
    
    if rebal_idx < (lookback_months - 1):
        skipped_count += 1
        continue
    
    window_indices = list(range(rebal_idx - (lookback_months - 1), rebal_idx + 1))
    actual_window_cols = [date_cols[i] for i in window_indices]
    window_dates = [dates[i] for i in window_indices]
    
    if len(actual_window_cols) != lookback_months:
        skipped_count += 1
        continue
    
    window_df = stocks_df[['PERMNO', 'Company_Ticker'] + actual_window_cols].copy()
    window_df = window_df[window_df['Company_Ticker'].notna()]
    
    valid_mask = window_df[actual_window_cols].notna().sum(axis=1) == lookback_months
    window_df = window_df[valid_mask]
    
    if len(window_df) < 20:
        skipped_count += 1
        continue
    
    returns = window_df[actual_window_cols].T
    returns.columns = window_df['PERMNO'].astype(str)
    
    timing_stats['io'].append(time.time() - t_io_start)

    stock_variance = returns.values.var(axis=0, ddof=1)
    min_variance = 1e-10
    valid_variance_mask = (stock_variance > min_variance) & np.isfinite(stock_variance)

    if valid_variance_mask.sum() < 2:
        skipped_count += 1
        continue

    valid_permnos = returns.columns[valid_variance_mask].tolist()
    returns = returns[valid_permnos]
    returns_np = returns.values
    window_df = window_df[window_df['PERMNO'].astype(str).isin(valid_permnos)].reset_index(drop=True)
    
    t0 = time.time()
    if GPU_AVAILABLE:
        returns_gpu = cp.asarray(returns_np)
        cov_array, shrinkage, cov_gpu = compute_covariance_gpu(returns_np, returns_gpu)
    else:
        cov_array, shrinkage, cov_gpu = compute_covariance_gpu(returns_np)
    timing_stats['cov'].append(time.time() - t0)
    
    cov = pd.DataFrame(cov_array, index=returns.columns, columns=returns.columns)
    
    t0 = time.time()
    if GPU_AVAILABLE:
        std_gpu = cp.sqrt(cp.diag(cov_gpu))
        std_gpu = cp.where(std_gpu < 1e-10, 1e-10, std_gpu)
        corr_gpu = cov_gpu / cp.outer(std_gpu, std_gpu)
        corr_gpu = cp.clip(corr_gpu, -1.0, 1.0)
    else:
        std = np.sqrt(np.diag(cov_array))
        std = np.where(std < 1e-10, 1e-10, std)
        corr_array = cov_array / np.outer(std, std)
        corr_array = np.clip(corr_array, -1.0, 1.0)
    timing_stats['corr'].append(time.time() - t0)
    
    t0 = time.time()
    if GPU_AVAILABLE:
        dist_gpu = get_correlation_distance_gpu(corr_gpu)
        eucl_dist_gpu = get_euclidean_distance_gpu(dist_gpu)
        eucl_dist_np = cp.asnumpy(eucl_dist_gpu)
        corr_array = cp.asnumpy(corr_gpu)
    else:
        corr_array = np.clip(corr_array, -1.0, 1.0)
        dist_np = np.sqrt(np.clip((1 - corr_array) / 2, 0.0, None))
        dist_np = np.nan_to_num(dist_np, nan=0.0, posinf=0.0, neginf=0.0)
        n = dist_np.shape[0]
        squared_norms = np.sum(dist_np ** 2, axis=1, keepdims=True)
        eucl_dist_np = np.sqrt(np.clip(squared_norms + squared_norms.T - 2 * np.dot(dist_np, dist_np.T), 0.0, None))
        eucl_dist_np = np.nan_to_num(eucl_dist_np, nan=1e-8, posinf=1e-8, neginf=1e-8)
    
    eucl_dist_np = np.nan_to_num(eucl_dist_np, nan=1e-8, posinf=1e-8, neginf=1e-8)
    from scipy.spatial.distance import squareform
    eucl_dist_condensed = squareform(eucl_dist_np, checks=False)
    eucl_dist_condensed = np.nan_to_num(eucl_dist_condensed, nan=1e-8, posinf=1e-8, neginf=1e-8)
    timing_stats['dist'].append(time.time() - t0)
    
    t0 = time.time()
    try:
        link = perform_single_linkage_clustering(eucl_dist_condensed)
    except Exception as e:
        print(f"⚠ Clustering failed for {rebal_str}: {e}, skipping")
        skipped_count += 1
        continue
    
    sort_ix = get_quasi_diag(link)
    sort_ix = returns.columns[sort_ix].tolist()
    timing_stats['cluster'].append(time.time() - t0)
    
    t0 = time.time()
    is_first_rebal = (rebal_date == target_rebalance_dates[0])
    hrp_weights = get_recursive_bisection(cov, sort_ix, debug=is_first_rebal)
    
    weight_sum = hrp_weights.sum()
    if abs(weight_sum - 1.0) > 1e-6:
        print(f"⚠ WARNING {rebal_str}: Weights sum to {weight_sum:.10f}, renormalizing...")
        hrp_weights = hrp_weights / weight_sum
    timing_stats['weights'].append(time.time() - t0)
    
    permno_to_ticker = dict(zip(window_df['PERMNO'].astype(str), window_df['Company_Ticker']))
    weights_df = pd.DataFrame({
        'PERMNO': hrp_weights.index,
        'Company_Ticker': [permno_to_ticker[p] for p in hrp_weights.index],
        rebal_str: hrp_weights.values
    })
    weights_list.append(weights_df)
    
    timing_stats['total'].append(time.time() - t_start)

if len(weights_list) > 0:
    all_weights = weights_list[0]
    for weights_df in weights_list[1:]:
        all_weights = all_weights.merge(weights_df, on=['PERMNO', 'Company_Ticker'], how='outer')
    all_weights = all_weights.sort_values('Company_Ticker').reset_index(drop=True)
    date_cols_in_df = [col for col in all_weights.columns if col not in ['PERMNO', 'Company_Ticker']]
    date_cols_sorted = sorted(date_cols_in_df)
    all_weights = all_weights[['PERMNO', 'Company_Ticker'] + date_cols_sorted]
    all_weights.to_csv(weights_output_path, index=False)
else:
    print("⚠ No weights computed!")
    all_weights = pd.DataFrame()

print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)
print(f"Mode: {'GPU (CUDA)' if GPU_AVAILABLE else 'CPU'}")
print(f"Total quarterly dates available: {len(quarterly_rebalance_dates)}")
print(f"Rebalance dates processed: {len(weights_list)}")
print(f"Skipped (insufficient history): {skipped_count}")
print(f"\nAverage timing per rebalance:")
print(f"  Data I/O & Prep: {np.mean(timing_stats['io'])*1000:.2f} ms  (CPU-only)")
print(f"  Covariance:      {np.mean(timing_stats['cov'])*1000:.2f} ms  ⚡ GPU")
print(f"  Correlation:     {np.mean(timing_stats['corr'])*1000:.2f} ms  ⚡ GPU")
print(f"  Distances:       {np.mean(timing_stats['dist'])*1000:.2f} ms  ⚡ GPU")
print(f"  Clustering:      {np.mean(timing_stats['cluster'])*1000:.2f} ms  (SciPy CPU)")
print(f"  Weight Calc:     {np.mean(timing_stats['weights'])*1000:.2f} ms  (CPU-only)")
print(f"  Total:           {np.mean(timing_stats['total'])*1000:.2f} ms")

gpu_time = np.sum(timing_stats['cov']) + np.sum(timing_stats['corr']) + np.sum(timing_stats['dist'])
cpu_time = np.sum(timing_stats['io']) + np.sum(timing_stats['cluster']) + np.sum(timing_stats['weights'])
other_time = np.sum(timing_stats['total']) - gpu_time - cpu_time

print(f"\n{'='*60}")
print("TIME BREAKDOWN")
print(f"{'='*60}")
print(f"  GPU Operations:  {gpu_time:.2f}s ({gpu_time/np.sum(timing_stats['total'])*100:.1f}%)")
print(f"  CPU Operations:  {cpu_time:.2f}s ({cpu_time/np.sum(timing_stats['total'])*100:.1f}%)")
print(f"  Other (overhead):{other_time:.2f}s ({other_time/np.sum(timing_stats['total'])*100:.1f}%)")
print(f"\nTotal runtime:    {np.sum(timing_stats['total']):.2f} seconds")
print(f"Speedup estimate: {np.sum(timing_stats['total']) / max(np.sum(timing_stats['total']), 1):.2f}x")
print(f"\n✓ Saved all HRP weights to {weights_filename}")

Processing all 180 quarterly rebalance dates


Processing quarterly rebalances: 100%|██████████| 180/180 [3:08:19<00:00, 62.77s/it]  



PERFORMANCE SUMMARY
Mode: GPU (CUDA)
Total quarterly dates available: 180
Rebalance dates processed: 177
Skipped (insufficient history): 3

Average timing per rebalance:
  Data I/O & Prep: 11.84 ms  (CPU-only)
  Covariance:      43605.62 ms  ⚡ GPU
  Correlation:     3.06 ms  ⚡ GPU
  Distances:       1757.44 ms  ⚡ GPU
  Clustering:      2486.02 ms  (SciPy CPU)
  Weight Calc:     15950.01 ms  (CPU-only)
  Total:           63834.34 ms

TIME BREAKDOWN
  GPU Operations:  8029.80s (71.1%)
  CPU Operations:  3265.27s (28.9%)
  Other (overhead):3.60s (0.0%)

Total runtime:    11298.68 seconds
Speedup estimate: 1.00x

✓ Saved all HRP weights to hrp_weights_all_scipy.csv


## Performance Architecture

**GPU-Accelerated Operations:**
- Covariance matrix computation (Ledoit-Wolf shrinkage)
- Correlation matrix calculation
- Distance matrix computations (correlation distance + Euclidean)
- Cumulative return calculations

**CPU Operations:**
- Hierarchical clustering (SciPy single-linkage with precomputed distances)
- Recursive bisection weight allocation
- Data I/O operations (loading, filtering, saving)

**Processing:** All quarterly rebalance dates from 1980 to present with 12-month rolling windows.

In [6]:
# Cell 5: Load HRP Weights and Prepare Benchmark Data

if 'weights_output_path' not in globals():
    raise RuntimeError("weights_output_path is not defined. Run the previous cell first.")
    
all_weights = pd.read_csv(weights_output_path)

rebal_dates_str = [col for col in all_weights.columns if col not in ['PERMNO', 'Company_Ticker']]
rebal_dates = pd.to_datetime(rebal_dates_str)

all_weights_long = all_weights.melt(
    id_vars=['PERMNO', 'Company_Ticker'],
    var_name='Rebalance_Date',
    value_name='Weight'
)
all_weights_long = all_weights_long.dropna(subset=['Weight'])

all_dates = dates

bench_df = df[df['PERMNO'].isna()]
vwretd_row = bench_df[bench_df['Company_Ticker'] == 'Value Weighted Benchmark']
ewretd_row = bench_df[bench_df['Company_Ticker'] == 'Equally Weighted Benchmark']

vwretd = pd.Series(vwretd_row.iloc[0][date_cols].values, index=dates, name='vwretd')
ewretd = pd.Series(ewretd_row.iloc[0][date_cols].values, index=dates, name='ewretd')

vwretd = pd.to_numeric(vwretd, errors='coerce')
ewretd = pd.to_numeric(ewretd, errors='coerce')

vwretd = vwretd.sort_index()
ewretd = ewretd.sort_index()

print(f"Loaded weights for {len(all_weights)} unique companies")
print(f"Rebalance dates: {len(rebal_dates)}")
print(f"Total weight entries (long format): {len(all_weights_long)}")

Loaded weights for 31491 unique companies
Rebalance dates: 177
Total weight entries (long format): 1105298


In [ ]:
# Cell 6: Compute HRP Portfolio Monthly Returns

port_returns = pd.Series(index=dates, dtype=float)
port_returns.name = 'HRP_CUDA_V2'

stocks_indexed = stocks_df.set_index('Company_Ticker')

for t_idx, t in enumerate(tqdm(dates, desc="Computing portfolio returns")):
    prev_rebal = max([rd for rd in rebal_dates if rd <= t], default=None)
    if prev_rebal is None:
        continue
    
    weights_df = all_weights_long[all_weights_long['Rebalance_Date'] == prev_rebal.strftime('%Y-%m-%d')]
    weights = weights_df.set_index('Company_Ticker')['Weight']
    
    col_name = date_cols[t_idx]
    
    available_tickers = weights.index.intersection(stocks_indexed.index)
    if len(available_tickers) == 0:
        port_returns[t] = np.nan
        continue
    
    ret_at_t = stocks_indexed.loc[available_tickers, col_name]
    weights_aligned = weights.loc[available_tickers]
    
    valid = ret_at_t.notna()
    if valid.sum() == 0:
        port_returns[t] = np.nan
        continue
    
    ret_valid = ret_at_t.loc[valid[valid].index]
    w_valid = weights_aligned.loc[valid[valid].index]
    w_valid /= w_valid.sum()
    
    port_returns[t] = (w_valid * ret_valid).sum()

port_returns = port_returns.dropna()
print(f"\n✓ Computed {len(port_returns)} monthly portfolio returns")

Computing portfolio returns:  45%|████▌     | 243/540 [00:22<00:28, 10.41it/s]

In [ ]:
# Cell 7: Compute Cumulative Returns

start_date = port_returns.index.min()
vwretd = vwretd[vwretd.index >= start_date]
ewretd = ewretd[ewretd.index >= start_date]
port_returns = port_returns[port_returns.index >= start_date]

vwretd = vwretd.dropna()
ewretd = ewretd.dropna()

common_index = port_returns.index.intersection(vwretd.index).intersection(ewretd.index)
port_returns = port_returns.loc[common_index]
vwretd = vwretd.loc[common_index]
ewretd = ewretd.loc[common_index]

if GPU_AVAILABLE:
    hrp_gpu = cp.asarray((1 + port_returns.values).astype(np.float64))
    vw_gpu = cp.asarray((1 + vwretd.values).astype(np.float64))
    ew_gpu = cp.asarray((1 + ewretd.values).astype(np.float64))
    
    cum_hrp = pd.Series(cp.asnumpy(cp.cumprod(hrp_gpu)), index=port_returns.index)
    cum_vw = pd.Series(cp.asnumpy(cp.cumprod(vw_gpu)), index=vwretd.index)
    cum_ew = pd.Series(cp.asnumpy(cp.cumprod(ew_gpu)), index=ewretd.index)
else:
    cum_hrp = (1 + port_returns).cumprod()
    cum_vw = (1 + vwretd).cumprod()
    cum_ew = (1 + ewretd).cumprod()

print("✓ Cumulative returns computed")
print(f"  HRP Final Value: {cum_hrp.iloc[-1]:.4f}")
print(f"  VW Final Value:  {cum_vw.iloc[-1]:.4f}")
print(f"  EW Final Value:  {cum_ew.iloc[-1]:.4f}")

In [ ]:
# Cell 8: Plot Performance Comparison

plt.figure(figsize=(14, 7))
plt.plot(cum_hrp.index, cum_hrp, label='HRP Portfolio (CUDA + SciPy)', linewidth=2.5, color='#0173B2')
plt.plot(cum_vw.index, cum_vw, label='Value Weighted Benchmark', linewidth=1.5, color='#DE8F05')
plt.plot(cum_ew.index, cum_ew, label='Equally Weighted Benchmark', linewidth=1.5, color='#029E73')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.title('Performance Comparison: CUDA-HRP (SciPy Single-Linkage) vs Benchmarks', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)
print(f"HRP Total Return:        {(cum_hrp.iloc[-1] - 1) * 100:.2f}%")
print(f"VW Benchmark Return:     {(cum_vw.iloc[-1] - 1) * 100:.2f}%")
print(f"EW Benchmark Return:     {(cum_ew.iloc[-1] - 1) * 100:.2f}%")
print(f"\nHRP Annualized Return:   {(cum_hrp.iloc[-1] ** (12 / len(cum_hrp)) - 1) * 100:.2f}%")
print(f"HRP Volatility:          {port_returns.std() * np.sqrt(12) * 100:.2f}%")
print(f"HRP Sharpe Ratio:        {(port_returns.mean() * 12) / (port_returns.std() * np.sqrt(12)):.2f}")

In [ ]:
# Cell 9: Visualize Weight Evolution (Top 20 Stocks)

weights_aggregated = all_weights_long.groupby(['Rebalance_Date', 'Company_Ticker'], as_index=False)['Weight'].sum()

weights_pivot = weights_aggregated.pivot(index='Rebalance_Date', columns='Company_Ticker', values='Weight')
weights_pivot.index = pd.to_datetime(weights_pivot.index)
weights_pivot = weights_pivot.sort_index()

ticker_avg_weights = weights_pivot.mean().sort_values(ascending=False)
top_20_tickers = ticker_avg_weights.head(20).index.tolist()

fig, ax = plt.subplots(figsize=(16, 9))

import matplotlib.cm as cm
colors = cm.tab20(np.linspace(0, 1, 20))

for i, ticker in enumerate(top_20_tickers):
    if ticker in weights_pivot.columns:
        ax.plot(weights_pivot.index, weights_pivot[ticker],
               alpha=0.8, linewidth=1.8, color=colors[i], label=ticker, marker='o', markersize=2)

ax.set_xlabel('Rebalance Date', fontsize=12)
ax.set_ylabel('Portfolio Weight', fontsize=12)
ax.set_title('Top 20 Stocks by Average Weight - Evolution Over Time', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)

overall_avg = ticker_avg_weights.head(20).mean()
ax.axhline(y=overall_avg, color='#CC0000', linestyle='--', linewidth=2.5,
           label=f'Top 20 Avg: {overall_avg:.4f}', alpha=0.9, zorder=100)

ax.legend(loc='upper left', fontsize=7, ncol=3, framealpha=0.95)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("TOP 20 STOCKS - WEIGHT EVOLUTION STATISTICS")
print("="*60)
print(f"Rebalance periods:       {len(weights_pivot)}")
print(f"Overall avg weight:      {overall_avg:.4f}")
print(f"\nTop 20 Stocks by Average Weight:")
print(f"{'Rank':<6}{'Ticker':<55}{'Avg Weight':<12}{'Max Weight':<12}{'Std Dev':<12}")
print("="*95)
for i, ticker in enumerate(top_20_tickers, 1):
    avg = weights_pivot[ticker].mean()
    max_w = weights_pivot[ticker].max()
    std = weights_pivot[ticker].std()
    print(f"{i:<6}{ticker:<55}{avg:<12.6f}{max_w:<12.6f}{std:<12.6f}")

In [ ]:
# Cell 10: Diagnostic Analysis - Weight Distribution & Data Quality

weights_csv_path = weights_output_path
df_weights = pd.read_csv(weights_csv_path)

print("="*80)
print("WEIGHTS DATA QUALITY DIAGNOSTIC")
print("="*80)

print(f"\n1. DATA STRUCTURE:")
print(f"   Total companies: {len(df_weights)}")
print(f"   Total columns: {len(df_weights.columns)}")
print(f"   Rebalance dates: {len(df_weights.columns) - 2}")

date_columns = [col for col in df_weights.columns if col not in ['PERMNO', 'Company_Ticker']]
print(f"   First rebalance: {date_columns[0] if date_columns else 'N/A'}")
print(f"   Last rebalance: {date_columns[-1] if date_columns else 'N/A'}")

print(f"\n2. DATA COMPLETENESS:")
total_cells = len(df_weights) * len(date_columns)
non_null_cells = df_weights[date_columns].notna().sum().sum() if date_columns else 0
sparsity = (total_cells - non_null_cells) / total_cells * 100 if total_cells else 0.0
print(f"   Total weight cells: {total_cells:,}")
print(f"   Non-null weights: {non_null_cells:,}")
print(f"   Sparsity (NaN %): {sparsity:.2f}%")

print(f"\n3. WEIGHT STATISTICS PER REBALANCE:")
if date_columns:
    weight_sums = df_weights[date_columns].sum(axis=0)
    portfolio_sizes = df_weights[date_columns].notna().sum(axis=0)
    print(f"   Min portfolio size: {portfolio_sizes.min()} stocks")
    print(f"   Max portfolio size: {portfolio_sizes.max()} stocks")
    print(f"   Avg portfolio size: {portfolio_sizes.mean():.1f} stocks")
    print(f"   Min weight sum: {weight_sums.min():.6f}")
    print(f"   Max weight sum: {weight_sums.max():.6f}")
    print(f"   Avg weight sum: {weight_sums.mean():.6f}")
else:
    print("   No rebalance dates found.")

print(f"\n4. WEIGHT DISTRIBUTION ANALYSIS:")
all_weights_flat = df_weights[date_columns].values.flatten() if date_columns else np.array([])
all_weights_flat = all_weights_flat[~np.isnan(all_weights_flat)]
if all_weights_flat.size > 0:
    print(f"   Min weight: {all_weights_flat.min():.8f}")
    print(f"   Max weight: {all_weights_flat.max():.8f}")
    print(f"   Mean weight: {all_weights_flat.mean():.8f}")
    print(f"   Median weight: {np.median(all_weights_flat):.8f}")
    print(f"   Std dev: {all_weights_flat.std():.8f}")
    extreme_high = (all_weights_flat > 0.01).sum()
    extreme_low = (all_weights_flat < 1e-6).sum()
    print(f"   Weights > 1%: {extreme_high} ({extreme_high/len(all_weights_flat)*100:.2f}%)")
    print(f"   Weights < 0.0001%: {extreme_low} ({extreme_low/len(all_weights_flat)*100:.2f}%)")
else:
    print("   No weight data available.")

print(f"\n5. REBALANCE DATES WITH ISSUES:")
problematic_dates = []
for date_col in date_columns:
    weight_sum = df_weights[date_col].sum()
    portfolio_size = df_weights[date_col].notna().sum()
    max_weight = df_weights[date_col].max()
    if abs(weight_sum - 1.0) > 0.01 or (portfolio_size > 0 and max_weight > 0.02) or portfolio_size < 20:
        problematic_dates.append({
            'Date': date_col,
            'Weight_Sum': weight_sum,
            'Portfolio_Size': portfolio_size,
            'Max_Weight': max_weight,
            'Max_Weight_Ticker': df_weights.loc[df_weights[date_col].idxmax(), 'Company_Ticker'] if portfolio_size > 0 else 'N/A'
        })
if problematic_dates:
    print(f"   Found {len(problematic_dates)} potentially problematic dates:")
    for issue in problematic_dates[:10]:
        print(f"   - {issue['Date']}: Sum={issue['Weight_Sum']:.4f}, Size={issue['Portfolio_Size']}, MaxWeight={issue['Max_Weight']:.4f} ({issue['Max_Weight_Ticker']})")
    if len(problematic_dates) > 10:
        print(f"   ... and {len(problematic_dates) - 10} more")
else:
    print("   ✓ No major issues detected!")

print(f"\n6. TOP 20 STOCKS DETAILED ANALYSIS:")
if date_columns:
    avg_weights = df_weights[date_columns].mean(axis=1)
    df_weights['Avg_Weight'] = avg_weights
    top_20_analysis = df_weights.nlargest(20, 'Avg_Weight')[['PERMNO', 'Company_Ticker', 'Avg_Weight']].copy()
    for idx, row in top_20_analysis.iterrows():
        ticker = row['Company_Ticker']
        weights_series = df_weights.loc[idx, date_columns]
        non_zero_periods = weights_series.notna().sum()
        first_appear = weights_series.first_valid_index()
        last_appear = weights_series.last_valid_index()
        max_weight = weights_series.max()
        print(f"   {ticker[:50]:<50} | Avg: {row['Avg_Weight']:.6f} | Periods: {non_zero_periods}/{len(date_columns)} | Max: {max_weight:.6f} | {first_appear} to {last_appear}")
else:
    print("   No rebalance data to analyze.")

print("\n" + "="*80)

## Understanding HRP Weight Behavior

**Normal HRP Characteristics:**
- **Sparse allocations**: Stocks rotate in/out based on data availability and filters
- **Correlation-driven concentration**: Market regimes affect concentration levels
- **Quarterly rebalancing**: Produces discrete jumps in weight allocation
- **Diversification**: Typically hundreds of positions with small individual weights

**Data Quality Indicators:**
- Weight sums should equal 1.0 (±0.01 tolerance)
- Portfolio sizes vary with data availability (typically 1000-6000 stocks)
- Individual weights usually < 1% in large universes
- Sparsity patterns reflect rolling window data requirements

## Summary

This notebook implements GPU-accelerated Hierarchical Risk Parity (HRP) with SciPy single-linkage clustering.

**Key Features:**
1. **GPU Acceleration**: Covariance, correlation, and distance computations run on CUDA
2. **Exact Clustering**: SciPy's `linkage(method='single')` with precomputed distance matrix
3. **Full Dataset**: Processes all quarterly rebalance dates (1980-present, 150+ windows)
4. **Robust Implementation**: Ledoit-Wolf covariance shrinkage with proper numpy array conversions

**Output Files:**
- `hrp_weights_all_scipy.csv`: HRP weights for all quarters
- Performance comparison plots vs CRSP benchmarks
- Detailed timing statistics for optimization analysis